#### Actividad - Proyecto práctico


> La actividad se desarrollará en grupos pre-definidos de 2-3 alumnos. Se debe indicar los nombres en orden alfabético (de apellidos). Recordad que esta actividad se corresponde con un 30% de la nota final de la asignatura. Se debe entregar entregar el trabajo en la presente notebook.
*   Alumno 1:
*   Alumno 2:
*   Alumno 3:
*   Alumno 4: 






---
## **PARTE 1** - Instalación y requisitos previos

> Las prácticas han sido preparadas para poder realizarse en el entorno de trabajo de Google Colab. Sin embargo, esta plataforma presenta ciertas incompatibilidades a la hora de visualizar la renderización en gym. Por ello, para obtener estas visualizaciones, se deberá trasladar el entorno de trabajo a local. Por ello, el presente dosier presenta instrucciones para poder trabajar en ambos entornos. Siga los siguientes pasos para un correcto funcionamiento:
1.   **LOCAL:** Preparar el enviroment, siguiendo las intrucciones detalladas en la sección *1.1.Preparar enviroment*.
2.  **AMBOS:** Modificar las variables "mount" y "drive_mount" a la carpeta de trabajo en drive en el caso de estar en Colab, y ejecturar la celda *1.2.Localizar entorno de trabajo*.
3. **COLAB:** se deberá ejecutar las celdas correspondientes al montaje de la carpeta de trabajo en Drive. Esta corresponde a la sección *1.3.Montar carpeta de datos local*.
4.  **AMBOS:** Instalar las librerías necesarias, siguiendo la sección *1.4.Instalar librerías necesarias*.


---
### 1.1. Preparar enviroment (solo local)



> Para preparar el entorno de trabajo en local, se han seguido los siguientes pasos:
1. En Windows, puede ser necesario instalar las C++ Build Tools. Para ello, siga los siguientes pasos: https://towardsdatascience.com/how-to-install-openai-gym-in-a-windows-environment-338969e24d30.
2. Instalar Anaconda
3. Siguiendo el código que se presenta comentado en la próxima celda: Crear un enviroment, cambiar la ruta de trabajo, e instalar librerías básicas.


```
conda create --name miar_rl python=3.8
conda activate miar_rl
cd "PATH_TO_FOLDER"
conda install git
pip install jupyter
```


4. Abrir la notebook con *jupyter-notebook*.



```
jupyter-notebook
```


---
### 1.2. Localizar entorno de trabajo: Google colab o local

In [ ]:
# ATENCIÓN!! Modificar ruta relativa a la práctica si es distinta (drive_root)
mount='/content/gdrive'
drive_root = mount + "/My Drive/08_MIAR/actividades/proyecto practico"

try:
  from google.colab import drive
  IN_COLAB=True
except:
  IN_COLAB=False

---
### 1.3. Montar carpeta de datos local (solo Colab)

In [ ]:
# Switch to the directory on the Google Drive that you want to use
import os
if IN_COLAB:
  print("We're running Colab")

  if IN_COLAB:
    # Mount the Google Drive at mount
    print("Colab: mounting Google drive on ", mount)

    drive.mount(mount)

    # Create drive_root if it doesn't exist
    create_drive_root = True
    if create_drive_root:
      print("\nColab: making sure ", drive_root, " exists.")
      os.makedirs(drive_root, exist_ok=True)

    # Change to the directory
    print("\nColab: Changing directory to ", drive_root)
    %cd $drive_root
# Verify we're in the correct working directory
%pwd
print("Archivos en el directorio: ")
print(os.listdir())

---
### 1.4. Instalar librerías necesarias

In [ ]:
if IN_COLAB:
  %pip install gym==0.17.3
  %pip install git+https://github.com/Kojoley/atari-py.git
  %pip install keras-rl2==1.0.5
  %pip install tensorflow==2.8
else:
  %pip install gym==0.17.3
  %pip install git+https://github.com/Kojoley/atari-py.git
  %pip install pyglet==1.5.0
  %pip install h5py==3.1.0
  %pip install Pillow==9.5.0
  %pip install keras-rl2==1.0.5
  %pip install Keras==2.2.4
  %pip install tensorflow==2.5.3
  %pip install torch==2.0.1
  %pip install agents==1.4.0

---
## **PARTE 2**. Enunciado

Consideraciones a tener en cuenta:

- El entorno sobre el que trabajaremos será [Acrobot_V1](https://gymnasium.farama.org/environments/classic_control/acrobot/) y el algoritmo que usaremos será _DQN_.

- Para nuestro ejercicio, el requisito mínimo será **alcanzar el objetivo de altura durante 100 episodios de test consecutivos**. Por ello, esta media de la recompensa se calculará a partir del código de test en la última celda del notebook.

Este proyecto práctico consta de tres partes:

1.   Implementar la red neuronal que se usará en la solución
2.   Implementar las distintas piezas de la solución DQN
3.   Justificar la respuesta en relación a los resultados obtenidos

**Rúbrica**: Se valorará la originalidad en la solución aportada, así como la capacidad de discutir los resultados de forma detallada. El requisito mínimo servirá para aprobar la actividad, bajo premisa de que la discusión del resultado sera apropiada.

IMPORTANTE:

* Si no se consigue una puntuación óptima, responder sobre la mejor puntuación obtenida.
* Para entrenamientos largos, recordad que podéis usar checkpoints de vuestros modelos para retomar los entrenamientos. En este caso, recordad cambiar los parámetros adecuadamente (sobre todo los relacionados con el proceso de exploración).
* Se deberá entregar unicamente el notebook y los pesos del mejor modelo en un fichero .zip, de forma organizada.
* Cada alumno deberá de subir la solución de forma individual.

---
## **PARTE 3**. Desarrollo y preguntas

#### Importar librerías

In [ ]:
import numpy as np
import gym
import tensorflow as tf

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.optimizers import Adam

from rl.agents.dqn import DQNAgent
from rl.policy import LinearAnnealedPolicy, EpsGreedyQPolicy
from rl.memory import SequentialMemory
from rl.callbacks import FileLogger, ModelIntervalCheckpoint


#### Configuración base

In [ ]:
ENV_NAME = 'Acrobot-v1'
SEED = 123
WINDOW_LENGTH = 1

# Variantes (activar según la propuesta)
USE_DOUBLE_DQN = False
USE_DUELING_DQN = False
DUELING_TYPE = 'avg'  # {'avg', 'max', 'naive'}

# Hiperparámetros principales
MEMORY_LIMIT = 100_000
WARMUP_STEPS = 1_000
TARGET_MODEL_UPDATE = 1_000  # pasos
GAMMA = 0.99
LEARNING_RATE = 1e-3
ANNEAL_STEPS = 50_000
EPS_MAX = 1.0
EPS_MIN = 0.05
EPS_TEST = 0.02
NB_STEPS_TRAIN = 200_000

env = gym.make(ENV_NAME)
np.random.seed(SEED)
tf.random.set_seed(SEED)
try:
    env.seed(SEED)
except Exception:
    pass

state_shape = env.observation_space.shape
nb_actions = env.action_space.n


In [ ]:
def set_global_seeds(seed=SEED):
    np.random.seed(seed)
    tf.random.set_seed(seed)

set_global_seeds(SEED)


1. Implementación de la red neuronal

La red para **Acrobot** no trabaja con píxeles, sino con el vector de estado continuo de 6 dimensiones. Por ello se utiliza una **MLP** sencilla que aproxima la función de valor de acción $Q_	heta(s,a)$, siguiendo la formulación de DQN [1]. En esta arquitectura, el estado se aplana y pasa por dos capas densas con activación ReLU; la salida produce un valor por acción discreta.

La elección de una MLP (en lugar de CNN) se justifica porque el espacio de observaciones no contiene estructura espacial, y el objetivo es aproximar $Q(s,a)$ de forma estable y eficiente [1], [2].


In [ ]:
# Red neuronal (MLP) para aproximar Q(s,a)
model = Sequential([
    Flatten(input_shape=(WINDOW_LENGTH,) + state_shape),
    Dense(128, activation='relu'),
    Dense(128, activation='relu'),
    Dense(nb_actions, activation='linear')
])

model.summary()


2. Implementación de la solución DQN

Se implementa DQN con: (i) **replay buffer** para romper correlaciones, (ii) política $\epsilon$-greedy con *annealing*, y (iii) *target network* opcionalmente actualizado cada cierto número de pasos, lo cual estabiliza el aprendizaje [1]. Las variantes **Double DQN** y **Dueling DQN** se activan con banderas, siguiendo las propuestas de estabilización y mejora del estimador de $Q$ descritas en [1].

La función objetivo se basa en el target de Bellman:

$$y(s,a)=r + \gamma \max_{a'} Q_{\theta^-}(s',a')$$

y la pérdida se minimiza típicamente con MSE o Huber:

$$\mathcal{L}(\theta)=\mathbb{E}\left[(y(s,a)-Q_{\theta}(s,a))^2\right].$$



In [ ]:
# Memoria de repetición y política epsilon-greedy con annealing
from pathlib import Path

memory = SequentialMemory(limit=MEMORY_LIMIT, window_length=WINDOW_LENGTH)
policy = LinearAnnealedPolicy(
    EpsGreedyQPolicy(),
    attr='eps',
    value_max=EPS_MAX,
    value_min=EPS_MIN,
    value_test=EPS_TEST,
    nb_steps=ANNEAL_STEPS,
)

# Agente DQN

dqn = DQNAgent(
    model=model,
    nb_actions=nb_actions,
    memory=memory,
    nb_steps_warmup=WARMUP_STEPS,
    target_model_update=TARGET_MODEL_UPDATE,
    gamma=GAMMA,
    policy=policy,
    enable_double_dqn=USE_DOUBLE_DQN,
    enable_dueling_network=USE_DUELING_DQN,
    dueling_type=DUELING_TYPE,
)

dqn.compile(Adam(learning_rate=LEARNING_RATE), metrics=['mae'])

# Entrenamiento
output_dir = Path('outputs')
checkpoints_dir = output_dir / 'checkpoints'
logs_dir = output_dir / 'logs'
checkpoints_dir.mkdir(parents=True, exist_ok=True)
logs_dir.mkdir(parents=True, exist_ok=True)

weights_path = checkpoints_dir / f'dqn_{ENV_NAME}_weights.h5f'
log_path = logs_dir / f'dqn_{ENV_NAME}_log.json'

callbacks = [
    ModelIntervalCheckpoint(str(weights_path), interval=10_000),
    FileLogger(str(log_path), interval=1_000)
]

dqn.fit(env, nb_steps=NB_STEPS_TRAIN, visualize=False, verbose=2, callbacks=callbacks)


In [ ]:
# Evaluación: 100 episodios consecutivos

dqn.load_weights(str(weights_path))

def evaluate_100_episodes(env, agent, nb_episodes=100):
    successes = 0
    rewards = []
    for ep in range(nb_episodes):
        obs = env.reset()
        state = obs[0] if isinstance(obs, tuple) else obs
        agent.reset_states()
        done = False
        total_reward = 0
        steps = 0
        while not done:
            action = agent.forward(state)
            step_out = env.step(action)
            if len(step_out) == 5:
                next_state, reward, done, truncated, _ = step_out
                done = done or truncated
            else:
                next_state, reward, done, _ = step_out
            state = next_state
            total_reward += reward
            steps += 1
        rewards.append(total_reward)
        max_steps = getattr(env, '_max_episode_steps', None)
        elapsed = getattr(env, '_elapsed_steps', steps)
        if max_steps is not None and elapsed < max_steps:
            successes += 1
    print(f"Éxitos: {successes}/{nb_episodes} ({successes/nb_episodes:.1%})")
    print(f"Recompensa media: {np.mean(rewards):.2f}")


evaluate_100_episodes(env, dqn, nb_episodes=100)


3. Justificación de los parámetros seleccionados y de los resultados obtenidos

**Justificación del diseño**

- **Arquitectura MLP**: Acrobot provee un estado continuo de baja dimensión; por tanto, una MLP es suficiente para aproximar $Q(s,a)$ sin necesidad de convoluciones [1].
- **Replay buffer**: reduce correlación temporal entre muestras y mejora la estabilidad del aprendizaje [1].
- **Target network**: desacopla la red objetivo de la red online, reduciendo oscilaciones [1].
- **Double/Dueling DQN (opcional)**: Double DQN reduce la sobreestimación de $Q$; Dueling separa el valor del estado y la ventaja por acción [1].

**Resultados (completar con la ejecución real)**

- Recompensa media en test (100 episodios): ________
- Éxitos (meta alcanzada): ________/100
- Observaciones sobre estabilidad, velocidad de convergencia y sensibilidad a hiperparámetros.

**Referencias (IEEE)**

[1] A. Géron, *Hands-On Machine Learning with Scikit-Learn, Keras & TensorFlow*, 3rd ed., 2025, ch. 18.

[2] R. S. Sutton and A. G. Barto, *Reinforcement Learning: An Introduction*, 2nd ed., MIT Press, 2020.


---